In [ ]:
# spustite si server pre deep learning, ktory ma nainstalovany tensorflow a keras 
# (File - Hub Control Panel ... Stop server, nasledne start ... a vyberte Jupyter Notbeook Deep Learning Stack )

# naimportujeme si potrebné knižnice a typy
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras import utils
import pandas as pd
from sklearn.metrics import classification_report

In [ ]:
# nacitame si data dostupne priamo v kerase - MNIST data, ide o rucne pisane cislice
((trainX, trainY), (testX, testY)) = mnist.load_data()

In [ ]:
# jeden konkretny priklad cislice 
plt.imshow(trainX[25], cmap='gray')

In [ ]:
# pre neuronove siete sa odporuca vstupy normalizovat - potrebujeme hodnoty pixelov (jas) normalizovat tak aby boli mensie ako 1, v tomto pripade teda na interval [0,1]
# kedze hodnoty jasu su od 0 do 255, zmenime typ na float a predelime ich /255
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

In [ ]:
# aj ked sa skala zmenila, obrazok vykresleny cez plt.imshow vidime ze vyzera rovnako, takze sme to zjavne neurobil chybu pri skalovani
plt.imshow(trainX[25], cmap='gray')

In [ ]:
# vytvorenie modelu ... sekvencne, povedzme ze urobime jednoduchu siet ktora pozostava z konvolucnej casti a doprednej casti siete
# akakolvek vrstva je prva, musi definovat input_shape vstupov, ktore sa pre nu budu pouzivat - u nas (28,28,1) pre obrazky 28*28 v 1 "farbe" (urovne jasu)
# ak chceme pouzit konvolucne vrstvy a pooling vrstvy, dame ich pred flatten vrstvu
# mozeme zakomponovat aj regularizaciu, napr. dropout
# Dense vrstvy su plne prepojene vrstvy pouzivane na klasifikaciu - representuju feed forward siet, postupne zmensujeme pocet neuronov vo vrstve a tak ucime kompaktnejsiu reprezentaciu
# posledna vrstva ma tolko neuronov, kolko je binarnych vystupov pre reprezentaciu toho, ktora cislica to je - cize 10 neuronov v nasom pripade, typ aktivacie softmax 
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(10, activation="softmax"))

# pomocou summary si vieme zobrazit strukturu modelu
model.summary()

In [ ]:
# skompilujeme model ... ide o akesi nastavenie celeho mechanizmu ucenia siete (samotne ucenie bude ako tradicne pomocou fit)
# optimizer je typ optimalizacneho algoritmu, pouzijeme oblubeny algoritmus Adam, s default nastaveniami (preto prazdna zatvorka pri volani)
# pre chybovu funkciu loss sa pre viac tried pozuiva categorical_crossentropy, ako metriku na urcenie kvality klasifikacie pouzijeme accuracy
# v pripade binarnej klasifikacie sa pouziva vystupna vrstva sigmoid (posledna Dense vrstva) a loss funkcia sa potom tu pozuije binary_crossentropy
adam = Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
# pre skompilovany model mozeme nasledne spustit proces ucenia pomocou fit
# mozeme pouzit validation_split a definovat kolko s trenovacej mnoziny bude pouzite samostatne ako validacna mnozina
# pripadne mozeme nastavit validacnu mnozinu samostatne (validation_data), alebo ju vobec nepouzit
# tiez musime cielove hodnoty trenovacej sady (trainY) pre potreby trenovania neuronovej siete zakodovat ako binarny vektor, kedze ich musime mapovat na softmax vrstvu s 10 neuronmi
# mozeme pouzit napr. utils funkciu kniznice keras to_categorical, vysledkom je napr. cislica 3 je zakodovane takto [0,0,0,1,0,0,0,0,0,0], cislica 7 zase [0,0,0,0,0,0,0,1,0,0] atd.
# dosiahneme to pouzitim utils.to_categorical(trainY, 10)
f = model.fit(trainX, utils.to_categorical(trainY, 10), epochs=2, batch_size=32, validation_split=0.1)

In [ ]:
# skusme predikciu konkretnej cislice, napr. index 32 (cize podmnozinu testX od 32 po 33 bez 33 - python)
# dostaneme aktivacne hodnoty na jednotlivych neuronoch vystupnej vrstvy - je to binarne zakodovana cislica, jedna hodnota je vyrazne vacsia, zodpoveda neuronu pre danu cislicu
some_prediction = model.predict(testX[32:33])
some_prediction

In [ ]:
# ako vyzeraju rozdiely v hodnotach predikcie si mozeme pozriet aj na grafe, ukaze nam poziciu najvacsej hodnoty
# podla toho kde je vidime, ze index pozicie je 3 -> cize cislica je 3
plt.bar(np.arange(10), some_prediction[0])

In [ ]:
# na porovnanie vykreslime obrazok daneho testovacieho prikladu, vidime ze skutocne ide o 3, predikcia bola uspesna
plt.imshow(testX[32], cmap='gray')

In [1]:
# vytvorime si predikcie na celej testX a prerobime ich na konkretnu hodnotu kategorie (cislicu) tak ze zistime argument (index) maximalnej hodnoty v ramci predikovaneho bin.vektora
y_predictions = model.predict(testX)
y_predicted_categories = np.argmax(y_predictions, axis = 1)


NameError: name 'model' is not defined

In [ ]:
# mozeme si vypisat predikovane hodnoty kategorii pre jednotlive prvky testX 

y_predicted_categories

In [ ]:
# vidime ze pre 32 zaznam to opat sedi, hodnota je 3
y_predicted_categories[32]

In [ ]:
# vytvorime confusion matrix na testovacej mnozine porovnanim predikovanych (y_predicted_ctaegories) a skutocnych hodnot y (testY) prikladov v testovacej mnozine
pd.crosstab(testY, y_predicted_categories)

In [ ]:
# mozeme vypisat aj klasifikacny report 
print(classification_report(testY, y_predicted_categories))